## Visualize prediction results
Feeds randomly selected images into a trained model and visualizes the results.
Follow this [guide](https://towardsdatascience.com/running-jupyter-notebook-in-google-cloud-platform-in-15-min-61e16da34d52) for setting up jupyter notebooks with gcloud compute.

In [ ]:
%matplotlib inline

# Create symlinks so can import them here
!ln -s ../scripts/draw_grid.py draw_grid.py
!ln -s ../src/imgcls/config config
!ln -s ../src/imgcls/modeling modeling

In [ ]:
import os
import json
import random
import numpy as np
from matplotlib import pyplot as plt
from draw_grid import draw_pred_vs_true
from config import get_cfg
import modeling
from detectron2.engine import DefaultPredictor
import cv2
import torch

In [ ]:
# Specify paths
data_json = '../data/synthetic/val.json'  # train.json, val.json or test.json
config_file = '../output/second_good_run/config.yaml'
model_weights = '../output/second_good_run/model_final.pth'

In [ ]:
GRID_SIZE = 4
cfg = get_cfg()
cfg.merge_from_file(config_file)
cfg.MODEL.WEIGHTS = os.path.abspath(model_weights)
cfg.INPUT.MIN_SIZE_TEST = 800 # apply resizing?
cfg.INPUT.MAX_SIZE_TEST = 800
cfg.INPUT.FORMAT = "BGR" # this is the case when using cv2.imread()

predictor = DefaultPredictor(cfg)
# docs here https://detectron2.readthedocs.io/modules/engine.html#detectron2.engine.defaults.DefaultPredictor

with open(data_json, 'r') as json_file:
    dataset_dicts = json.load(json_file)

In [ ]:
# TODO: overlay image
def visualize_preds():
    """Gets a random sample from 'data_json' feeds it into the model and compares predictions to truth labels"""
    d = random.sample(dataset_dicts, 1)[0]
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    plt.imshow(img)
    plt.show()
    output = predictor(img)
    predictions = output['pred'].cpu().round().numpy().astype(int)
    truth_labels = np.array(d['label'])
    print("labels: " + str(truth_labels))
    print("preds: " + str(predictions))
    img = draw_pred_vs_true(img,
                            predictions.reshape(GRID_SIZE, GRID_SIZE), 
                            truth_labels.reshape(GRID_SIZE, GRID_SIZE))
   
    plt.imshow(img)
    plt.show()

In [ ]:
# run this block several times to get a qualitative sense of how well the model is doing
visualize_preds()

In [ ]:
# clean up symlinks
!rm draw_grid.py modeling config ../src/imgcls/config/config ../src/imgcls/modeling/modeling